## Course: TM10007 - Machine learning
Editors: Lishia Vergeer, Amy Roos, Maaike Pruijt, Hilde Roording.

Description: The aim of this code is to predict the tumor grade of glioma’s(high or low) before surgery, 
based on features extracted from a combination of four MRI images: 
T2-weighted, T2-weighted FLAIR and T1-weighted before and after injection of contrast agent.

#### Import packages

In [137]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds

# Import code
from brats.load_data import load_data

# Performance 
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import seaborn


# Pipeline and gridsearch
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

#preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# scaler
from sklearn.preprocessing import RobustScaler

#Machine learning classifiers
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import ShuffleSplit

import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsRegressor



#### Load data

In [138]:
data_brats = load_data()

# Convert to dataframe
X = pd.DataFrame(data_brats)

# Print data 
print(f'The number of samples in data_brats: {len(X.index)}')
print(f'The number of columns in data_brats: {len(X.columns)}')

The number of samples in data_brats: 167
The number of columns in data_brats: 725


#### Split data in X and y
Split in X (data) and y (label)

In [139]:
# split column label from dataset X
y = X.pop('label')

#### Split data in train and test set
This function creates a panda dataframe and splits the data into test and train components.
This is done with test_size variable and the function train_test_split from the sklearn module.
Returns a train set with the data of 80% and a test set of 20% of the subjects.



In [140]:
# Split data in train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)  


#### Preprocessing : 

In [141]:
# infinity to NaN
X_train[X_train==np.inf]=np.nan
X_test[X_test==np.inf]=np.nan

# non-numeric features to NaN
X_train = X_train.replace(['#DIV/0!'], np.nan)
X_train = X_train.apply(pd.to_numeric, errors='coerce')

X_test  = X_test.replace(['#DIV/0!'], np.nan)
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# If the total number of NaN observations in a column are greater than 40%, delete the entire column.
perc = 40.0
min_count = int(((100-perc)/100)*X_train.shape[0] + 1)
X_train_drop = X_train.dropna(axis=1, thresh=min_count)
X_labels = X_train_drop.keys()

X_test_drop = X_test[X_labels]
print(X_test_drop.shape)
print(X_train_drop.shape)

# fill the NaN observations.
data_fill_train = X_train_drop.fillna(X_train_drop.mean()) 
data_fill_test = X_test_drop.fillna(X_test_drop.mean()) 

print(data_fill_test)

(34, 704)
(133, 704)
              VOLUME_ET  VOLUME_NET  VOLUME_ED  VOLUME_TC  VOLUME_WT  \
ID                                                                     
TCGA-14-1794      65093       52012      83517     117105     200622   
TCGA-06-0164      19875       17388      72888      37263     110151   
TCGA-76-6661      77776       49074      45957     126850     172807   
TCGA-HT-8114       8755      168606      11325     177361     188686   
TCGA-06-0142      50084        8759      70998      58843     129841   
TCGA-FG-A4MT        119       62001      25407      62120      87527   
TCGA-08-0389      14157        7328      55793      21485      77278   
TCGA-02-0087       3852       42877      59124      46729     105853   
TCGA-06-0162      87754       15960      47101     103714     150815   
TCGA-CS-6665          0       59424      48440      59424     107864   
TCGA-12-0616      40616        7774      21238      48390      69628   
TCGA-02-0046      35721        4212     103

#### Scale features

In [142]:
# robustscaler
scaler = RobustScaler()
scaler.fit(data_fill_train)
X_train_scaled = scaler.transform(data_fill_train)
X_test_scaled = scaler.transform(data_fill_test)

#### Transform features

In [143]:
# Perform a PCA
pca = decomposition.PCA(n_components=2)
pca.fit(X_train_scaled) 
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


#### Cross validation

In [144]:
# Create a cross-validation object

cv = KFold(n_splits=5, shuffle= True, random_state = 1)

X = X_train_pca

y = y_train.values
y = np.where(y=='GBM', 1, y)
y = np.where(y=='LGG', 2, y)
y = y.tolist()

classifiers = (KNeighborsRegressor(), RandomForestClassifier() )

for cls in classifiers:

    r_2s = cross_val_score(cls, X, y, scoring = 'accuracy', cv=cv)
    avg_r2 = np.mean(r_2s)

    print(r_2s)
    print(avg_r2)


[0.85185185 0.66666667 0.88888889 0.92307692 0.65384615]
0.7968660968660969
[0.96296296 0.74074074 0.92592593 0.80769231 0.73076923]
0.8336182336182336


#### Classifier

In [145]:
# Construct classifiers
clsfs = [RandomForestClassifier(n_estimators=1, random_state=42),
         RandomForestClassifier(n_estimators=5, random_state=42),
         RandomForestClassifier(n_estimators=200, random_state=42)]


